In [ ]:
!pip install tensorflow
!pip install opencv-python

In [3]:
import cv2
import tensorflow as tf
import numpy as np

In [5]:
model = tf.keras.models.load_model("emotion_model.keras")

C:\Users\Acer\anaconda3\envs\tf-gpu\lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 36 variables whereas the saved optimizer has 70 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [6]:


# Class labels (adjust if your dataset has different classes)
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Load the Haar Cascade face detector
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Extract the face ROI
        face_roi = gray[y:y + h, x:x + w]

        # Preprocess for model
        face_roi = cv2.resize(face_roi, (48, 48))
        face_roi = face_roi.astype('float32') / 255.0
        face_roi = np.expand_dims(face_roi, axis=-1)  # Add channel dimension
        face_roi = np.expand_dims(face_roi, axis=0)   # Add batch dimension

        # Get prediction
        prediction = model.predict(face_roi, verbose=0)
        label = class_labels[np.argmax(prediction)]
        confidence = np.max(prediction)

        # Display label and confidence on top-right of rectangle
        cv2.putText(
            frame, f'{label} ({confidence*100:.1f}%)',
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8, (0, 255, 0), 2
        )

    # Display the output
    cv2.imshow('Emotion Detection', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()
